In [5]:
import sys
sys.path.append(".")
import unittest
import numpy as np
import matplotlib.pyplot as plt
from susie.timing_data import TimingData
from susie.ephemeris import Ephemeris
from scipy.optimize import curve_fit
test_epochs = np.array([0, 294, 298, 573])
test_mtts = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_mtts_err = np.array([0.00043, 0.00028, 0.00062, 0.00042])
test_tra_or_occ = np.array(['tra','occ','tra','occ'])
test_P_linear = 1.0914223408652188 # period linear
test_P_err_linear = 9.998517417992763e-07 # period error linear
test_T0_linear =  -6.734666196939187e-05 # conjunction time
test_T0_err_linear = 0.0003502975050463415 # conjunction time error

test_P_quad = 1.0914215464474404 #period quad
test_P_err_quad = 9.150815726215122e-06 # period err quad
test_dPdE = 2.7744598987630543e-09#period change by epoch
test_dPdE_err = 3.188345582283935e-08#period change by epoch error
test_T0_quad = -1.415143555084551e-06 #conjunction time quad
test_T0_err_quad = 0.00042940561938685084#conjunction time err quad

test_observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_uncertainties= np.array([0.00043, 0.00028, 0.00062, 0.00042])



In [6]:
epoch_data = np.array([0, 294, 298, 573])
mid_transit_time_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
mid_transit_times_uncertainties_data = np.array([0.00043, 0.00028, 0.00062, 0.00042])
# STEP 2: Break data up into epochs, mid transit times, and error
epochs = epoch_data - np.min(epoch_data)
mid_transit_times = mid_transit_time_data - np.min(mid_transit_time_data)
mid_transit_times_err = mid_transit_times_uncertainties_data
# STEP 2.5 (Optional): Make sure the epochs are integers and not floats
epochs = epochs.astype('int')

In [7]:
transit_times_obj1 = TimingData('jd', epochs, mid_transit_times, mid_transit_times_err, test_tra_or_occ, time_scale='tdb')

In [24]:
ephemeris_obj1 = Ephemeris(transit_times_obj1)

In [25]:
linear_model_data = ephemeris_obj1.get_model_ephemeris('linear')
linear_model_data

{'period': 1.09142234035412,
 'period_err': 4.018260635819698e-07,
 'conjunction_time': -6.728390649943598e-05,
 'conjunction_time_err': 0.00013969441566829874,
 'model_type': 'linear',
 'model_data': array([-6.72839065e-05,  3.20878101e+02,  3.25243790e+02,  6.25384934e+02])}

In [26]:
def linear_model(test_x, test_P_linear, test_T0_linear):
    """Linear model function."""
    return test_P_linear * test_x + test_T0_linear


popt, pcov = curve_fit(linear_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
    'period': popt[0],
    'period_err': unc[0],
    'conjunction_time': popt[1],
    'conjunction_time_err': unc[1]
}
return_data

{'period': 1.0914223408652188,
 'period_err': 9.998517417992763e-07,
 'conjunction_time': -6.734666196939187e-05,
 'conjunction_time_err': 0.0003502975050463415}

In [27]:
def quad_model(test_x, test_dPdE, test_P_quad, test_T0_quad):
    return 0.5*test_dPdE*test_x*test_x + test_P_quad*test_x + test_T0_quad
popt, pcov = curve_fit(quad_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
            'conjunction_time': popt[2],
            'conjunction_time_err': unc[2],
            'period': popt[1],
            'period_err': unc[1],
            'period_change_by_epoch': popt[0],
            'period_change_by_epoch_err': unc[0],
        }
return_data

{'conjunction_time': -1.415143555084551e-06,
 'conjunction_time_err': 0.00042940561938685084,
 'period': 1.0914215464474404,
 'period_err': 9.150815726215122e-06,
 'period_change_by_epoch': 2.7744598987630543e-09,
 'period_change_by_epoch_err': 3.188345582283935e-08}

In [28]:
quadratic_model_data = ephemeris_obj1.get_model_ephemeris('quadratic')
quadratic_model_data

{'period': 1.091421537944888,
 'period_err': 1.3369529628386657e-06,
 'conjunction_time': -8.102185298292572e-07,
 'conjunction_time_err': 0.00019627998726801882,
 'period_change_by_epoch': 2.8048128182651046e-09,
 'period_change_by_epoch_err': 4.3654905738095855e-09,
 'model_type': 'quadratic',
 'model_data': array([-8.10218530e-07,  3.20878053e+02,  3.25243742e+02,  6.25385001e+02])}

In [36]:
##linear fit
P = test_P_linear
E = test_epochs
T0 = 0
tra_or_occ =  np.array([0,1,0,1])
tra = np.round(P * E +T0)
print(tra)
occ = np.round(P * E +(T0+0.5*P))
print(occ)
lin_fit = np.array([0.         , 3.21423879e+02, 325.24385758, 6.25930712e+02 ])


[  0. 321. 325. 625.]
[  1. 321. 326. 626.]


In [40]:
##quad fit
P = test_P_quad
E = test_epochs
T0 = 0
dPdE = 0
tra_q = np.round(0.5*dPdE*E*E + P*E + 0)
print(tra_q)
occ_q = np.round((1 + 0.5*P) + P*E + 0.5*dPdE*E*E)
print(occ_q)
quad_fit = np.array([0,321,325,626])

[  0. 321. 325. 625.]
[  2. 322. 327. 627.]


In [42]:
##linear model uncertainites
T0_err_l = 0.23692091722329203
P_err_l =  0.0006807480614626216
E = test_epochs
tra = (np.sqrt((T0_err_l**2) + ((test_epochs**2)*(P_err_l**2))))
print(tra)
occ = (np.sqrt(((T0_err_l**2) + (0.5 * (P_err_l**2))) + ((test_epochs**2)*(P_err_l**2))))
print(occ)
result = np.array([0.23692092, 0.31014149, 0.31190525, 0.45638284 ])

[0.23692092 0.31014112 0.31190525 0.45638259]
[0.23692141 0.31014149 0.31190562 0.45638284]


In [43]:
##quad model uncertainite
T0_err_q = 0.34674308676945004
P_err_q = 0.0023688537695923505
dPdE_err = 7.743357776955459e-06
E = test_epochs
tra = (np.sqrt((T0_err_q**2) + ((E**2)*(P_err_q**2)) + ((1/4)*(E**4)*(dPdE_err**2))))
print(tra)
occ = (np.sqrt(((T0_err_q**2) + (0.5 * (P_err_q**2))) + ((E**2)*(P_err_q**2)) + ((1/4)*(E**4)*(dPdE_err**2))))
print(occ)
result = np.array([ 0.34674309, 0.84691127, 0.85834968, 1.89170591 ])

[0.34674309 0.84690961 0.85834968 1.89170516]
[0.34674713 0.84691127 0.85835132 1.89170591]


In [17]:
## calc quad ephem
period_change_by_epoch = test_dPdE
epochs = test_epochs
period = test_P_quad
conjunction_time = test_T0_quad
((0.5*period_change_by_epoch*(epochs**2)) + (period*epochs) + conjunction_time)

array([-1.41514356e-06,  3.20878053e+02,  3.25243743e+02,  6.25385000e+02])

In [18]:
##calc lin ephem
period = test_P_linear
conjunction_time = test_T0_linear
((period*epochs) + conjunction_time)

array([-6.73466620e-05,  3.20878101e+02,  3.25243790e+02,  6.25384934e+02])

In [19]:
#calc chi linear
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data = [-6.73466620e-05, 3.20878101e+02, 3.25243790e+02, 6.25384934e+02]
uncertainties =  np.array([0.00043, 0.00028, 0.00062, 0.00042])
np.sum(((observed_data - model_data)/uncertainties)**2)

0.29406284565290114

In [20]:
#calc chi quad
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data = np.array([-1.41514356e-06,  3.20878053e+02,  3.25243743e+02,  6.25385000e+02])
uncertainties =  np.array([0.00043, 0.00028, 0.00062, 0.00042])
np.sum(((observed_data - model_data)/uncertainties)**2)

0.20766342879185204

In [21]:
## calc bic lin
chi_squared= 0.29406284565290114
num_params=2

chi_squared + (num_params*np.log(4))


3.0666515678926825

In [22]:
## calc bic quad
chi_squared= 0.20766342879185204
num_params=3

chi_squared + (num_params*np.log(4))

4.3665465121515235

In [23]:
linear_bic = 3.0666515678926825
quadratic_bic = 4.3665465121515235
linear_bic - quadratic_bic

-1.299894944258841